In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("data/Quote_data.csv")

In [6]:
df.head()

,Quote,Author,Tags,revised_tags
0,“Be yourself; everyone else is already taken.”,Oscar Wilde,"attributed-no-source, be-yourself, gilbert-per...","['attributed-no-source', 'be-yourself', 'hones..."
1,“You've gotta dance like there's nobody watchi...,William W. Purkey,"dance, heaven, hurt, inspirational, life, love...","['dance', 'heaven', 'hurt', 'inspirational', '..."
2,“Be the change that you wish to see in the wor...,Mahatma Gandhi,"action, change, inspirational, misattributed-t...","['action', 'change', 'inspirational', 'philoso..."
3,“Darkness cannot drive out darkness: only ligh...,"Martin Luther King Jr., A Testament of Hope: ...","darkness, drive-out, hate, inspirational, ligh...","['darkness', 'hate', 'inspirational', 'light',..."
4,“Live as if you were to die tomorrow. Learn as...,Mahatma Gandhi,"carpe-diem, education, inspirational, learning","['carpe-diem', 'education', 'inspirational', '..."


In [7]:
df.columns

Index(['Quote', 'Author', 'Tags', 'revised_tags'], dtype='object')

In [8]:
df.drop(['Tags', 'revised_tags'],axis=1,inplace=True)

In [9]:
df.head()

,Quote,Author
0,“Be yourself; everyone else is already taken.”,Oscar Wilde
1,“You've gotta dance like there's nobody watchi...,William W. Purkey
2,“Be the change that you wish to see in the wor...,Mahatma Gandhi
3,“Darkness cannot drive out darkness: only ligh...,"Martin Luther King Jr., A Testament of Hope: ..."
4,“Live as if you were to die tomorrow. Learn as...,Mahatma Gandhi


In [10]:
df.shape

(29355, 2)

In [11]:
# df = df[:15000]

In [12]:
df.shape

(29355, 2)

In [ ]:
quotes = df['Quote'].copy()

In [14]:
# Remove null values
quotes = quotes.dropna()

In [15]:
quotes = quotes.str.lower()

In [16]:
# Remove all punctuation marks (.,!?"' etc.)
import string
quotes =  quotes.str.translate(str.maketrans('', '', string.punctuation))

In [17]:
 # Smart filtering: Remove extreme outliers
quote_lengths = quotes.str.split().str.len()
mean_len = quote_lengths.mean()
std_len = quote_lengths.std()

# Keep quotes within 2 standard deviations (removes noise)
quotes = quotes[(quote_lengths >= 4) & (quote_lengths <= 35)]
quotes = quotes.reset_index(drop=True)

print(f"After filtering: {len(quotes):,} quotes")
print(f"Average quote length: {quotes.str.split().str.len().mean():.1f} words")

After filtering: 19,267 quotes
Average quote length: 18.1 words


In [18]:
quotes.head()

0         “be yourself everyone else is already taken”
1    “youve gotta dance like theres nobody watching...
2    “be the change that you wish to see in the world”
3    “darkness cannot drive out darkness only light...
4    “live as if you were to die tomorrow learn as ...
Name: Quote, dtype: object

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
vocab_size = 8000
tokenizer = Tokenizer(num_words=vocab_size,oov_token = '<UNK>', filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(quotes)

In [20]:
word_index = tokenizer.word_index
# print(word_index)

In [21]:
actual_vocab_size = vocab_size +1# +1 for padding token at index 0
print(f"Actual vocabulary size: {actual_vocab_size}")

Actual vocabulary size: 8001


In [22]:
# actual_vocab_size = min(vocab_size, len(word_index)) + 1

In [23]:
len(word_index)

26395

In [24]:
list(word_index.items())[:10]

[('<UNK>', 1),
 ('the', 2),
 ('to', 3),
 ('is', 4),
 ('of', 5),
 ('you', 6),
 ('a', 7),
 ('and', 8),
 ('in', 9),
 ('that', 10)]

In [25]:
sequences = tokenizer.texts_to_sequences(quotes)

In [26]:
# Purpose: Create input-output pairs for supervised learning.
X = []
y = []
max_sequence_len = 35  # Limit sequence length to 50 words max

for seq in sequences:
    # Only use sequences with reasonable length
    if len(seq) > 1:
        for i in range(1, min(len(seq), max_sequence_len)):
            input_seq = seq[:i]
            output_seq = seq[i]
            X.append(input_seq)
            y.append(output_seq)

In [27]:
X[:10]

[[361],
 [361, 120],
 [361, 120, 253],
 [361, 120, 253, 227],
 [361, 120, 253, 227, 4],
 [361, 120, 253, 227, 4, 416],
 [2142],
 [2142, 1923],
 [2142, 1923, 579],
 [2142, 1923, 579, 41]]

In [28]:
y[:10]

[120, 253, 227, 4, 416, 6168, 1923, 579, 41, 304]

In [29]:
max_len = max(len(x) for x in X) + 1

In [30]:
max_len

35

In [31]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_padded = pad_sequences(X, maxlen=max_len, padding='pre')

In [32]:
y = np.array(y)

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [34]:
embedding_dim = 200
lstm_units = 200

In [35]:
from tensorflow.keras.layers import GRU

model = Sequential([
    Embedding(actual_vocab_size, 128, mask_zero=True),
    GRU(256, dropout=0.2, recurrent_dropout=0.2),
    Dropout(0.3),
    Dense(actual_vocab_size, activation='softmax')
])


I0000 00:00:1769347666.271606      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [ ]:
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy

# Optimizer with custom learning rate
optimizer = Adam(learning_rate=0.001, clipnorm=1.0)  # Gradient clipping prevents explosion

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',
            SparseTopKCategoricalAccuracy(k=3, name='top_3_acc'),
            SparseTopKCategoricalAccuracy(k=5, name='top_5_acc')
            ]
)

print("\n" + "="*70)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [37]:
# Save best model during training
checkpoint = ModelCheckpoint(
    'best_quote_model.keras',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Early stopping - but with more patience
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,  # More patience for complex model
    restore_best_weights=True,
    verbose=1
)

# Reduce learning rate when stuck
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=0.00001,
    verbose=1
)

In [ ]:
import time

# Train for more epochs - P100 can handle it
start_time = time.time()
history = model.fit(
    X_padded, y,
    validation_split=0.2,  # 20% validation
    epochs=100,  # Will stop early if converging
    batch_size=128,  # Optimal for P100
    callbacks=[checkpoint, early_stopping,reduce_lr],
    verbose=1
)

training_time = (time.time() - start_time) / 60
print(f"\nTraining completed in {training_time:.1f} minutes")

In [ ]:
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
best_val_acc = max(history.history['val_accuracy'])
best_epoch = history.history['val_accuracy'].index(best_val_acc) + 1

print("\n" + "="*70)
print("📈 TRAINING RESULTS")
print("="*70)
print(f"Total Epochs:           {len(history.history['loss'])}")
print(f"Best Epoch:             {best_epoch}")
# print(f"Training Time:          {training_time:.1f} minutes")
print(f"\nFinal Training Accuracy:    {final_train_acc:.2%}")
print(f"Final Validation Accuracy:  {final_val_acc:.2%}")
print(f"Best Validation Accuracy:   {best_val_acc:.2%}")
print(f"\nFinal Training Loss:        {final_train_loss:.4f}")
print(f"Final Validation Loss:      {final_val_loss:.4f}")
print(f"\nOverfitting Gap:            {(final_train_acc - final_val_acc):.2%}")

if final_train_acc - final_val_acc < 0.10:
    print("✅ EXCELLENT! Very low overfitting")
elif final_train_acc - final_val_acc < 0.20:
    print("✅ GOOD! Acceptable overfitting")
elif final_train_acc - final_val_acc < 0.30:
    print("⚠️ MODERATE overfitting")
else:
    print("❌ HIGH overfitting - consider more regularization")

In [ ]:
# Save the model
model.save('quote_generator_model_3.keras')

# Save the tokenizer
import pickle

with open('tokenizer_3.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

# Save important parameters
import json
model_params = {
    'max_len': int(max_len),
    'actual_vocab_size': int(actual_vocab_size),
    'embedding_dim': embedding_dim,
    'lstm_units': lstm_units,
    'training_samples': len(X),
    # 'training_time_minutes': float(training_time),
    'final_train_accuracy': float(final_train_acc),
    'final_val_accuracy': float(final_val_acc),
    'best_val_accuracy': float(best_val_acc),
    'final_train_loss': float(final_train_loss),
    'final_val_loss': float(final_val_loss),
    'epochs_trained': len(history.history['loss']),
    'best_epoch': int(best_epoch)
}

with open('model_params_3.json', 'w') as f:
    json.dump(model_params, f, indent=2)

# Save training history
history_dict = {
    'accuracy': [float(x) for x in history.history['accuracy']],
    'val_accuracy': [float(x) for x in history.history['val_accuracy']],
    'loss': [float(x) for x in history.history['loss']],
    'val_loss': [float(x) for x in history.history['val_loss']]
}

with open('training_history_3.json', 'w') as f:
    json.dump(history_dict, f, indent=2)

print("✅ quote_generator_model_3.keras")
print("✅ best_quote_generator_model_3.keras (saved during training)")
print("✅ tokenizer_3.pkl")
print("✅ model_params_3.json")
print("✅ training_history.json")


print("✅ Model saved successfully!")
print("✅ Tokenizer saved successfully!")
print("✅ Parameters saved successfully!")

In [ ]:
import numpy as np
import pickle
import json
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the model
# model = load_model('model/quote_generator_model_3.keras')
model  = load_model('model/best_quote_model.keras')

# Load the tokenizer
with open('model/tokenizer_3.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# Load parameters
with open('json/model_params_3.json', 'r') as f:
    model_params = json.load(f)

max_len = model_params['max_len']
word_index = tokenizer.word_index

print("✅ Everything loaded successfully!")
print(f"📊 Vocabulary size: {model_params['actual_vocab_size']}")
print(f"📏 Max sequence length: {max_len}")

In [ ]:
# Create reverse index ONCE (outside function for speed)
reverse_word_index = {v: k for k, v in tokenizer.word_index.items()}

def generate_quote(seed_text, next_words=10, temperature=0.8, top_k=50):
    """
    Generate text with improved quality
    
    Args:
        seed_text: Starting text
        next_words: Number of words to generate
        temperature: Randomness (0.5=safe, 1.0=balanced, 1.5=creative)
        top_k: Only sample from top k predictions (prevents bad words)
    """
    result = seed_text.lower()
    
    for _ in range(next_words):
        # Tokenize current text
        token_list = tokenizer.texts_to_sequences([result])[0]
        token_list = pad_sequences([token_list], maxlen=max_len, padding='pre')
        
        # Get predictions
        predictions = model.predict(token_list, verbose=0)[0]
        
        # ============================================
        # FILTER BAD TOKENS
        # ============================================
        # Remove <PAD> and <UNK>
        predictions[0] = 0  # <PAD>
        predictions[1] = 0  # <UNK>
        
        # Renormalize
        if predictions.sum() == 0:
            break
        predictions = predictions / predictions.sum()
        
        # ============================================
        # TOP-K SAMPLING (Better quality)
        # ============================================
        # Get top k predictions
        top_k_indices = np.argsort(predictions)[-top_k:]
        top_k_probs = predictions[top_k_indices]
        
        # Apply temperature to top-k only
        top_k_probs = np.power(top_k_probs, 1/temperature)
        top_k_probs = top_k_probs / top_k_probs.sum()
        
        # Sample from top-k
        predicted_id = np.random.choice(top_k_indices, p=top_k_probs)
        
        # ============================================
        # GET WORD
        # ============================================
        output_word = reverse_word_index.get(predicted_id, "")
        
        # Safety check
        if not output_word or output_word in ["<UNK>", "<PAD>"]:
            break
            
        result += " " + output_word
    
    return result


# ============================================
# Test with different temperatures
# ============================================
print("\n" + "="*70)
print("🎨 GENERATED QUOTES (Multiple Temperatures)")
print("="*70)

seeds = ["life is", "love is", "be yourself", "the only", "success comes"]

for seed in seeds:
    print(f"\n{'='*70}")
    print(f"Seed: '{seed}'")
    print(f"{'='*70}")
    
    # Conservative (more predictable)
    quote1 = generate_quote(seed, next_words=10, temperature=0.5, top_k=30)
    print(f"🔵 Safe (T=0.5):     {quote1}")
    
    # Balanced
    quote2 = generate_quote(seed, next_words=10, temperature=0.8, top_k=50)
    print(f"🟢 Balanced (T=0.8): {quote2}")
    
    # Creative (more random)
    quote3 = generate_quote(seed, next_words=10, temperature=1.2, top_k=100)
    print(f"🟡 Creative (T=1.2): {quote3}")